# Welcome to Scheduler!

### Getting started
To begin please choose a method for loading data. Option A lets you load an excel spreadsheet with your own data. Option B loads some sample data for you to see the process.

First we ought to get into the correct directory:

In [1]:
cd ../scheduler

/Users/gcallah/GitProjects/Scheduler/scheduler


In case you haven't installed the pandas or xlrd packages, the next cell will install them:

In [2]:
!pip3 install pandas
!pip3 install xlrd

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Option A

*So let's cut the A and B options and just use A!*

Here we will be loading from the excel file 'my_data.xl' which you can download, edit with spreadsheet software and reupload here. When you have done this please run the cell below. 

In [7]:

import pandas as pd
from IPython.display import display 

df = pd.read_excel('my_data.xlsx')

prof_df = df.iloc[:,0:4]
prof_df['prof_courses'] = prof_df['prof_courses'].str.replace(', ',',')
rooms_df = df.iloc[:,4:6]
rooms_df.dropna(inplace=True)
rooms_df['room_capacity'] = rooms_df['room_capacity'].astype(u'int8')
rooms_df['rooms'] = rooms_df['rooms'].astype('str')
courses_df = df.iloc[:,6:]
courses_df.dropna(inplace=True)

professors = []
prof_info = {}
for index, row in prof_df.iterrows():
        data = {}
        prof = row['professor']
        professors.append(prof)
        courses = row['prof_courses'].split(',')
        data['courses'] = courses
        data['start_time'] = row['prof_start_time']
        data['end_time'] = row['prof_end_time']
        prof_info[prof] = data
        
rooms = []
room_capacities = {}
for _, row in rooms_df.iterrows():
    room = row['rooms']
    room_capacities[room] = row['room_capacity']
    rooms.append(room)

courses = []
course_no_students = {}
course_mins = {}
course_days_weekly = {}
for _, row in courses_df.iterrows():
    course = row['course']
    courses.append(course)
    course_no_students[course] = int(row['course_no_students'])
    course_mins[course] = int(row['course_mins'])
    course_days_weekly[course] = int(row['course_days_weekly'])
print("Data loaded successfully!")

Data loaded successfully!


Before continuing you may check that the courses, professors and classrooms listed below are correct.

In [5]:
print('\n'.join(professors))

John Smith
Lisa Jones
Tim Simpson
Simon Valinski
Lisa Mullen
Jamir Abdullah'
Mike Williams
Rachel Smith
Gregg Woods
Chu Yen
Peter Parker
Elizabeth Walker
Brian K. Dickson


In [5]:
print('\n'.join(rooms))

655
666
745a
745b
433
201
115a
115b


In [6]:
print('\n'.join(courses))

physics
astrophysics
caculus 1
calculus 4
biology 1
biology 2
molecular biology
biochemistry
chemistry


### Option B
Run the cell below (shift + enter) to create the sample data we need. This will take data from datamaker.py (which is the file  where you can add/edit/remove classes, teachers, classrooms; and exports that data into a file called sample_data.txt to be used.

In [3]:
import json
import datamaker

datamaker.make_data()

Now we load the created data into our program (run the cell below)

In [4]:
input_file = "sample_data.txt"
with open(input_file,'r') as f:
    data = json.load(f)
    
professors = data['professors']
prof_info = data['prof_info']
rooms = data['rooms']
room_capacities = data['room_capacities']
courses = data['courses']
course_no_students = data['course_no_students']
course_mins = data['course_mins']
course_days_weekly = data['course_days_weekly']

The data is ready to use. 

Running the cell below shows us a list of available professors

In [5]:
print('\n'.join(professors))

John Smith
Lisa Jones
Mike Williams
Tim Simpson
Rachel Smith
Gregg Woods
Simon Valinski
Chu Yen
Peter Parker
Lisa Mullen
Elizabeth Walker
Brian K. Dickson
Jamir Abdullah


Similarly, run the cell below to see a list of rooms

In [6]:
print('\n'.join(rooms))

655
666
745a
745b
433
201
115a
115b


And finally do the same for the list of courses

In [7]:
print('\n'.join(courses))

physics
chemistry
biochemistry
biology 1
biology 2
molecular biology
calculus 1
calculus 4
astrophysics


The scheduler program will take the courses shown above and find a professor, classroom and time for them.

### Scheduling the Teachers, Courses and Classrooms 

Run the following two cells

In [8]:
from teachercourse_csp import assigner

In [9]:
def time_formatter(course, start_time):
    hs,ms = start_time
    end = hs*6 + ms//10 + course_mins[course]//10
    end_time = (end//6, (end - (end // 6) * 6) * 10)
    he,me = end_time
    output = '{:0>2}'.format(hs)+':'+'{:0>2}'.format(ms)+' to ' + '{:0>2}'.format(he)+':'+ '{:0>2}'.format(me) 
    return output

Now we are ready to compute the class schedule and display it below


In [44]:
user_data = (professors,prof_info,rooms,room_capacities,courses,course_no_students,course_mins,course_days_weekly)
full_schedule = assigner(user_data)
weekdays = ['mon','tues','wed','thur','fri']

columns = ['Day','Course','Professor', 'Room','Period']
df_out = pd.DataFrame(None, columns=columns)
for day in weekdays:
    schedule = full_schedule[day]
    print(day.upper())
    print('-----')
    for var,val in schedule.items():
        course,professor = var
        room,start_time = val
        df_inc = {'Day':day,'Course':[course],'Professor':[professor], 'Room':[room],'Period':[time_formatter(course,start_time)]}
        df_inc = pd.DataFrame.from_dict(df_inc)
        df_out = pd.concat([df_out,df_inc],ignore_index=True)
    print('')

MON
-----

TUES
-----

WED
-----

THUR
-----

FRI
-----



In [38]:
display(df_out)

,Day,Course,Professor,Room,Period
0,mon,physics,Brian K. Dickson,745b,14:00 to 15:00
1,mon,calculus 4,Brian K. Dickson,433,16:00 to 17:00
2,mon,biology 1,Peter Parker,745b,09:00 to 10:30
3,mon,chemistry,John Smith,115b,09:30 to 11:00
4,tues,physics,Brian K. Dickson,115b,13:00 to 14:00
5,tues,biochemistry,Peter Parker,433,09:30 to 11:00
6,wed,astrophysics,Simon Valinski,655,13:30 to 14:30
7,wed,calculus 4,Brian K. Dickson,745b,16:30 to 17:30
8,wed,biology 1,Peter Parker,745b,12:00 to 13:30
9,wed,biology 2,Rachel Smith,745b,11:00 to 12:00
